# Batch Process Source HEC-RAS Models
Generate FIM Libraries for multiple HEC-RAS Models

In [1]:
%load_ext autoreload
%autoreload 2

from scripts.processing import *

### `Parameters`

In [2]:
stac_endpoint = "https://stac2.dewberryanalytics.com"  # STAC API endpoint
stac_collection = "ebfe-12040101_WestForkSanJacinto"  # Collection name

source_models_dir = r"Z:\collections\ebfe-12040101_WestForkSanJacinto\source_models"
submodels_dir = r"Z:\collections\ebfe-12040101_WestForkSanJacinto\submodels"
library_dir = r"Z:\collections\ebfe-12040101_WestForkSanJacinto\library"
db_path = r"Z:\collections\ebfe-12040101_WestForkSanJacinto\ripple.db"

### Get Working Models

In [3]:
models_files = get_models_from_stac(stac_endpoint, stac_collection)
# or get based on some geographic unit
# models = ["Baxter"]
# download_model_files(models_files, source_models_dir)
model_keys = list(models_files.keys())

Total 439 models in this collection
Total 262 filtered models.


### Load Conflation Information to Database

In [4]:
load_conflation(model_keys, source_models_dir, db_path)

Conflation loaded to Z:\collections\ebfe-12040101_WestForkSanJacinto\ripple.db from .conflation.json files


### Update `network_to_id` Table in Database
Build a modified network by skipping over reaches eclipsed by adjacent reaches

In [5]:
update_network(db_path)

Updated 454 reaches successfully.
CPU times: total: 1.03 s
Wall time: 19.8 s


### Get Working Reaches

In [7]:
print(model_keys)
reaches = get_reaches_by_models(db_path, model_keys)

dict_keys(['WHITE OAK CREEK', 'WFSJR 097', 'WFSJR 094', 'WFSJR 092', 'WFSJR 090', 'WFSJR 089', 'WFSJR 087', 'WFSJR 086', 'WFSJR 085', 'WFSJR 084', 'WFSJR 082', 'WFSJR 080', 'WFSJR 078', 'WFSJR 076', 'WFSJR 075', 'WFSJR 071', 'WFSJR 068', 'WFSJR 065', 'WFSJR 064', 'WFSJR 058', 'WFSJR 057', 'WFSJR 055', 'WFSJR 053', 'WFSJR 050', 'WFSJR 048', 'WFSJR 046', 'WFSJR 045', 'WFSJR 042', 'WFSJR 041', 'WFSJR 039', 'WFSJR 038', 'WFSJR 034', 'WFSJR 033', 'WFSJR 030', 'WFSJR 026', 'WFSJR 024', 'WFSJR 023', 'WFSJR 022', 'WFSJR 019', 'WFSJR 018', 'WFSJR 017', 'WFSJR 016', 'WFSJR 012', 'WFSJR 009', 'WFSJR 007', 'WFSJR 006', 'WFSJR 004', 'WFSJR 003', 'WFSJR 002', 'WFSJR 001', 'WEST SANDY CREEK', 'WEST ROCKY CREEK', 'WEST ROCKY BRANCH', 'WEST GUM BRANCH', 'WEST CEDAR CREEK', 'WELCH BRANCH', 'TRIBUTARY 6', 'TRIBUTARY 5', 'SCOTT BRANCH 01', 'ROCKY CREEK', 'ROBINSON CREEK', 'ROBINSON BRANCH', 'POLECAT BRANCH', 'NEELY SPRING BRANCH', 'MOCK BRANCH', 'MILL BRANCH', 'MCGARY CREEK', 'LUCAS BRANCH', 'LEE BRANCH',

ProgrammingError: parameters are of unsupported type

### Run `extract_submodel`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches = execute_step(reaches, "extract_submodel", db_path, source_models_dir, submodels_dir)
if len(failed_reaches) + len(not_accepted_reaches) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `create_ras_terrain`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches = execute_step(reaches, "create_ras_terrain", db_path, source_models_dir, submodels_dir)
if len(failed_reaches) + len(not_accepted_reaches) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `create_model_run_normal_depth`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches = execute_step(reaches, "create_model_run_normal_depth", db_path, source_models_dir, submodels_dir)
if len(failed_reaches) + len(not_accepted_reaches) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `run_incremental_normal_depth`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches = execute_step(reaches, "run_incremental_normal_depth", db_path, source_models_dir, submodels_dir)
if len(failed_reaches) + len(not_accepted_reaches) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `run_known_wse` for Network

In [ ]:
execute_kwse_for_network([(reach[0], None) for reach in reaches if reach[1] is None], submodels_dir, db_path, False)

## Merge Libraries and Rating Curves

In [ ]:
move_fims_to_library(submodels_dir, library_dir)

In [ ]:
load_all_rating_curves(library_dir, db_path)